In [30]:
# import library 
import wfdb
import pywt
import seaborn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras import Model, Input
from tensorflow.python.keras.layers import LSTM, Dropout, Dense,Attention, multiply
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import *
from wfdb import processing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm



In [31]:
"""
custom function
"""
# Use the GQRS detection algorithm and correct the peaks
def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()


# compute_hr: This is the compute the heart rate
# sig_len: 
# qrs_ins: 
# fs: frequency
def compute_hr(sig_len, qrs_ins, fs):
    heart_rate = np.full(sig_len, np.nan, dtype = "float32")

    if len(qrs_inds) < 2:
        return heart_rate

    for i in range(0, len(qrs_inds) - 2):
            a = qrs_inds[i]
            b = qrs_inds[i + 1]
            c = qrs_inds[i + 2]
            rr = (b - a) * (1.0 / fs) * 1000
            hr = 60000.0 / rr
            heart_rate[b + 1: c + 1] = hr

            heart_rate[qrs_inds[-1]:] = heart_rate[qrs_inds[-1]]

            return heart_rate
        
# compute_hr_custom: This is the compute the custom hr 
# sig_len: 
# qrs_ins: 
# fs: frequency

def compute_respiratory(r_peak_1, r_peak_2, fs):
     a = r_peak_1
     b = r_peak_2
     rr = (b - a) * (1.0 / fs)
     hr = 60 / rr

     return hr
    
# dont know wtf for this
def denoise(data):
   
    coeffs = pywt.wavedec(data=data, wavelet='db5', level = 9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs

    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    rdata = pywt.waverec(coeffs=coeffs, wavelet='db5')
    return rdata


def plotHeatMap(Y_test, Y_pred):
    con_mat = confusion_matrix(Y_test, Y_pred)
    # 绘图
    plt.figure(figsize=(4, 5))
    seaborn.heatmap(con_mat, annot=True, fmt='.20g', cmap='Blues')
    plt.ylim(0, 5)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()
    
    
    
def buildModel():
    newModel = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(300, 1)),
        # 第一个卷积层, 4 个 21x1 卷积核
        tf.keras.layers.Conv1D(filters=4, kernel_size=21, strides=1, padding='SAME', activation='tanh'),
        # 第一个池化层, 最大池化,4 个 3x1 卷积核, 步长为 2
        tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='SAME'),
        # 第二个卷积层, 16 个 23x1 卷积核
        tf.keras.layers.Conv1D(filters=16, kernel_size=23, strides=1, padding='SAME', activation='relu'),
        # 第二个池化层, 最大池化,4 个 3x1 卷积核, 步长为 2
        tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='SAME'),
        # 第三个卷积层, 32 个 25x1 卷积核
        tf.keras.layers.Conv1D(filters=32, kernel_size=25, strides=1, padding='SAME', activation='tanh'),
        # 第三个池化层, 平均池化,4 个 3x1 卷积核, 步长为 2
        tf.keras.layers.AvgPool1D(pool_size=3, strides=2, padding='SAME'),
        # 第四个卷积层, 64 个 27x1 卷积核
        tf.keras.layers.Conv1D(filters=64, kernel_size=27, strides=1, padding='SAME', activation='relu'),
        # 打平层,方便全连接层处理'
        tf.keras.layers.Flatten(),
        # 全连接层,128 个节点 转换成128个节点
        tf.keras.layers.Dense(128, activation='relu'),
        # Dropout层,dropout = 0.2
        tf.keras.layers.Dropout(rate=0.2),
        # 全连接层,5 个节点
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    return newModel

def plotHeatMap(Y_test, Y_pred):
    con_mat = confusion_matrix(Y_test, Y_pred)
    # 绘图
    plt.figure(figsize=(4, 5))
    seaborn.heatmap(con_mat, annot=True, fmt='.20g', cmap='Blues')
    plt.ylim(0, 5)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()


In [51]:
  
"""
Step 1: Load sample data for 1..10 infants

heart_data: list of heart rate ECG sample data
resp_data: list of respiratory data

"""
# Constant files
ecg_file = './DataSet/files/picsdb/1.0.0/infant{}_ecg'
resp_file = './DataSet/files/picsdb/1.0.0/infant{}_resp'
numbers_infants = 3
sampfrom = 0
sampto = 1000

for i in range(1, numbers_infants):
    print("Loading infants", i, "...")

    ecg_record = wfdb.rdrecord(ecg_file.format(i), sampfrom = sampfrom, sampto = sampto)
    ecg_annotation = wfdb.rdann(ecg_file.format(i), 'qrsc', sampfrom = sampfrom, sampto = sampto)

    resp_record = wfdb.rdrecord(resp_file.format(i), sampfrom = sampfrom, sampto = sampto)
    resp_annotation = wfdb.rdann(resp_file.format(i), 'resp', sampfrom = sampfrom, sampto = sampto)

#     wfdb.plot_wfdb(record=ecg_record, annotation=ecg_annotation,
#                title='Record from ECG',
#                time_units='minutes')
    
#     wfdb.plot_wfdb(record=resp_record, annotation=resp_annotation,
#                title='Record from respiratory',
#                time_units='minutes')
        

    """DEBUG"""
    # Display record dictionary
    #     display(ecg_record.__dict__)
#     display(resp_record.__dict__)

    
    # Convert to p_signal
    ecg_p_signal = ecg_record.p_signal
    resp_p_signal = resp_record.p_signal

    # ECG Signal Grpah
    plt.plot(ecg_p_signal)
    plt.title("ECG Signal")
    plt.show()
    
    # Respiratory Signal Grpah
    plt.plot(resp_p_signal)
    plt.title("Respiratory Signal")
    plt.show()

    """ 
    STEP 1
    REFINING THE GRAPH - ECG
    """
    # Use the GQRS algorithm to detect QRS locations in the first channel
    qrs_inds = processing.qrs.gqrs_detect(sig = ecg_record.p_signal, fs = ecg_record.fs)
    
    # Correct the peaks shifting them to local maxima
    min_bpm = 20
    max_bpm = 230
    min_gap = ecg_record.fs * 60 / min_bpm

    denoise_record_p_signal = denoise(ecg_record.p_signal.flatten())
    denoise_record_fs = ecg_record.fs 
    # denoise_record_fs = denoise(ecg_record.fs) 

    # Use the maximum possible bpm as the search radius
    peaks_hr(sig = ecg_record.p_signal, peak_inds=qrs_inds, fs=ecg_record.fs,
         title="GQRS peak detection on record 100")
    
    search_radius = int(denoise_record_fs * 60 / max_bpm)
    corrected_peak_inds = processing.peaks.correct_peaks(denoise_record_p_signal, 
                                                         peak_inds=qrs_inds,
                                                         search_radius=search_radius, 
                                                         smooth_window_size=150)

    print('Corrected GQRS detected peak indices:', sorted(corrected_peak_inds))
    peaks_hr(sig = ecg_record.p_signal, peak_inds = sorted(corrected_peak_inds), fs=ecg_record.fs,
         title="Corrected GQRS peak detection on sampledata/" + str(sampto))

    
    """ 
    STEP 2
    Calculating the heart rate
    Corelation: 
    heart_rate = by checking
    """
    
    # Heart rate and respiratory rate
    heart_rate = processing.compute_hr(len(ecg_record.p_signal), sorted(corrected_peak_inds), ecg_record.fs)
    breathing_rate = resp_record.p_signal.flatten()
    
    ecg_samp_interval = 1 / ecg_record.fs
    resp_samp_interval = 1 / resp_record.fs

    print("ECG slp frequecy:", ecg_record.fs, "Hz")
    print("ECG slp interval every:", ecg_samp_interval, "sec")

    print("RESP slp frequecy:" , resp_record.fs, "Hz")
    print("RESP slp interval every:", ecg_samp_interval, "sec")
    print("")
    
    time_ecg = np.arange(ecg_record.p_signal.shape[0]) * ecg_samp_interval
    time_resp = np.arange(resp_record.p_signal.shape[0]) * resp_samp_interval
    
    heart_data = {
        'Infant:': i,
        'Heart_rate': heart_rate,
        'Time(sec)': time_ecg,
    }

    resp_data = {
        'Infant:': i,
        'Heart_rate': breathing_rate,
        'Time(sec)': time_resp, 
    }

    
    # dropna() -> Remove NaN
    heart_df = pd.DataFrame.from_dict(heart_data).dropna()
    
    columns = [column for column in heart_df.columns if resp_df[column].nunique()==1]
    resp_df.drop(columns=columns)

    final_resp_df = pd.DataFrame.from_dict(resp_data)
    

    a = np.intersect1d(heart_df.columns, final_resp_df.columns)
    
    df3 = pd.merge(resp_df, final_resp_df, how='inner'，left_on='Time(sec)', right_on='Time(sec)')

    print("heart_df", heart_df)
    print("resp_df", resp_df, final_resp_df)
    print("df3", df3)

    """
    Train data set
    """
    X_train, X_test, Y_train, Y_test = train_test_split(heart_df, resp_df, test_size=0.2, random_state=42)
    
    
    """
    Data Modelling 1: SVM
    """
    # show unclassified data
#     plt.scatter(X_train, Y_train)
#     plt.show()

#     print("X_train", len(X_train))
#     print("X_test", len(X_test))
#     print("y_train", len(Y_train))
#     print("y_test", len(Y_test))
    
#     clf = svm.SVC(kernel='linear', C=1.0)
#     clf.fit(X_train, Y_train)
    
#     print("")


print("Trained All infants data")


SyntaxError: invalid character '，' (U+FF0C) (414439895.py, line 132)

In [ ]:
from tensorflow.keras import Model, Input

 
tf.keras.models.fit(X_train, Y_train, validation_split = "linear")  # validation_split 训练集所占比例
